In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import os

In [4]:
data = pd.read_table('Problem I - rsids.txt')

In [5]:
data.head()

,Annotation,Chr,Position,Genotypes
0,rs121434297,1,11855218,AA|0.4581
1,rs387907088,12,65722364,TT|0.4953
2,rs121434561,19,48624501,GG|0.4929
3,rs1109867,2,220083453,TG|0.7395
4,rs2231164,4,89015857,TT|0.8277


In [6]:
data.dtypes

Annotation    object
Chr           object
Position       int64
Genotypes     object
dtype: object

In [7]:
[data.loc[data[col]=='--', :].head() for col in data.columns if(col!='Position')]

[Empty DataFrame
 Columns: [Annotation, Chr, Position, Genotypes]
 Index: [], Empty DataFrame
 Columns: [Annotation, Chr, Position, Genotypes]
 Index: [], Empty DataFrame
 Columns: [Annotation, Chr, Position, Genotypes]
 Index: []]

In [8]:
data.columns = data.columns.str.replace('Annotation', 'rsID')
data.head()

,rsID,Chr,Position,Genotypes
0,rs121434297,1,11855218,AA|0.4581
1,rs387907088,12,65722364,TT|0.4953
2,rs121434561,19,48624501,GG|0.4929
3,rs1109867,2,220083453,TG|0.7395
4,rs2231164,4,89015857,TT|0.8277


In [9]:
#Separate the call score from Genometypes
genome_type, genome_score = [], []

def create_type_score(data):
    type_, score = data.split('|')
    genome_type.append(type_)
    genome_score.append(score)
    

In [10]:
#Create two columns with Genome Type and Scores
#Removing the column 'Genotypes'
def create_cols(data):
    global genome_type, genome_score
    if(len(genome_score)==0 and len(genome_type)==0):
        
        #Apply the function for each row of the column Genotypes
        list(map(create_type_score, data['Genotypes']))
        data['Genome Type'], data['Genome Call Score'] =  genome_type, genome_score
        
        return
    else:
        genome_type, genome_score = [], []
        

In [11]:
if('Genotypes' in data.columns):
    create_cols(data)
    data = data.drop(['Genotypes'], axis=1)
    print(data.head())
list(map(len, [genome_type, genome_score]))

          rsID Chr   Position Genome Type Genome Call Score
0  rs121434297   1   11855218          AA            0.4581
1  rs387907088  12   65722364          TT            0.4953
2  rs121434561  19   48624501          GG            0.4929
3    rs1109867   2  220083453          TG            0.7395
4    rs2231164   4   89015857          TT            0.8277


[3000, 3000]

In [12]:
data.head()

,rsID,Chr,Position,Genome Type,Genome Call Score
0,rs121434297,1,11855218,AA,0.4581
1,rs387907088,12,65722364,TT,0.4953
2,rs121434561,19,48624501,GG,0.4929
3,rs1109867,2,220083453,TG,0.7395
4,rs2231164,4,89015857,TT,0.8277


## A
Find unique rsIDs and chromosome and location with genotype keeping in mind the following
things:

1) While removing duplication consider keeping observations with higher call rates

2) Remove no calls represented as ‘--’ but if all repeated observation are no calls keep one
observation with no call (to imply that rsID has no call in the data).

In [13]:
def plotting(data, var_X, var_y, rs_id):
    title = var_X + ' Versus ' + var_y
    plt.scatter(data[var_X], data[var_y])
    plt.title(title)
    plt.xlabel(var_X)
    plt.ylabel(var_y)
    #plt.show()
    plt.savefig(os.getcwd() + "/Plots/" + str(rs_id) + "_" + title + '.png')

In [14]:
#Find pattern in the data
def analyze_duplicates(data, rs_id):
    #Is the variation located on the same 
    #chromosome for all the duplicates.
    plotting(data, 'Chr', 'Position', rs_id)
    plotting(data, 'Chr', 'Genome Type', rs_id)
    plotting(data, 'Genome Type', 'Genome Call Score', rs_id)
    

In [ ]:
#Resultant Data
result_df = pd.DataFrame(columns=data.columns)

#Group the rows by 'rsID'
data_grp = data.groupby('rsID')

#Sort each group in descending order of Genome Call Score
for grp, df in data_grp:
    #Need to consider the groups as independent data frames
    df = df.reset_index()
    
    #An additional(redundant) variable 'index' will be created
    df = df.drop(['index'], axis=1)
    
    df = df.sort_values(by='Genome Call Score', ascending=False)
    
    #Keep only the first row amongst the duplicate rows
    result_df = pd.concat([result_df, df.iloc[:1,:]], axis=0)
    
    #If multiple entries in df ====> Duplicate entries
    

In [ ]:
#List Comprehension is 
[analyze_duplicates(df, grp) for grp, df in data_grp if(df.shape[0]>1)]

In [192]:
len(list(data_grp.groups.keys()))

1524

In [193]:
data.shape

(3000, 5)

In [179]:
result_df.head()

,rsID,Chr,Position,Genome Type,Genome Call Score
0,1:207754848-gataa-g,1,207754848,II,0.8803
0,rs10007810,4,41554364,AG,0.4823
1,rs10037858,5,156766708,GG,0.9145
1,rs10108270,8,4190793,AC,0.6523
0,rs10136766,14,106232585,GG,0.4365


,rsID,Chr,Position,Genome Type,Genome Call Score
